CS RESEARCH PAPER RECOMMENDER

Working on google colab as I will be needing faster processing and more RAM for this project.

Dataset Used: "Arxiv Dataset" from kaggle.com.

Initial processing was done in a kaggle notebook so I will not be including that part here. The result data from the initial processing is used here for futher processing. 

This notebook is not for showcasing the development of the recommender system. It is to show the different experimentations and decisions involved in the development of the project. 

In [2]:
from google.colab import drive        # Mounting google drive to google colab to import the datasets.
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
import numpy as np  # Importing necessary libraries
import pandas as pd
import re
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from datetime import datetime
import nltk
from sklearn.metrics.pairwise import *
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from numba import jit
nltk.download("stopwords")
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.decomposition import TruncatedSVD
import files

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stop = stopwords.words()

Saving the stopwords.words() into a variable and then using it in 
"(' '.join(e.lower() for e in sentance.split() if e.lower() not in stop)" improved the performance of the data cleaning function dramatically.

In this case, initially I was getting an estimated time of 63 hours for the data cleaning function to run but after adding 'stop' as a variable, the estimated time reduced to 1 hour 10 minutes. (for 210k rows)

In [ ]:
def decontracted(phrase):
    '''
    Converts words with " ' " to full form.
    '''
    # specific
    phrase = re.sub(r"won't", "will not", phrase)  
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase


def text_to_vector(sentance):       
    """
    Converts text to final required vector.
    """
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stop)
    stemmer = SnowballStemmer("english")
    stemmed_text = []
    for word in sentance.split():
        word = stemmer.stem(word)
        stemmed_text.append(word)
    sentance = " ".join(stemmed_text)
    sentance= tfidf.transform([sentance])
    return sentance


def recommendations(sentance,vectors_tfidf):
    """
    Gives recommendations after calculating similarity.
    """
    sentance = text_to_vector(sentance)
    sentance = pca.transform(sentance)
    similarities = []
    for i in tqdm(range(vectors_tfidf.shape[0])):
        similarities.append(paired_cosine_distances(sentance,vectors_tfidf[i].reshape(-1,2000))[0])
    similarities= np.array(similarities).argsort()
    top_rec = similarities[:10]
    for item in top_rec:
        print(data_info.iloc[item,:]['title'])
        print("\n")
    

These functions were built during the first iteration of the development of the project. 
It was observed that taking the user input and converting it to vector to give recommendations was taking about 24 seconds to compute per query.
24 seconds to give recommendations was impractical in the case of building a web application for the project.
Therefore I had to find a work-around to reduce the time.

In [ ]:
sentance = 'Generalized LSTM-based End-to-End Text-Independent Speaker Verification  The increasing amount of available data and more affordable hardware\nsolutions have opened a gate to the realm of Deep Learning (DL). Due to the\nrapid advancements and ever-growing popularity of DL, it has begun to invade\nalmost every field, where machine learning is applicable, by altering the\ntraditional state-of-the-art methods. While many researchers in the speaker\nrecognition area have also started to replace the former state-of-the-art\nmethods with DL techniques, some of the traditional i-vector-based methods are\nstill state-of-the-art in the context of text-independent speaker verification\n(TI-SV). In this paper, we discuss the most recent generalized end-to-end\n(GE2E) DL technique based on Long Short-term Memory (LSTM) units for TI-SV by\nGoogle and compare different scenarios and aspects including utterance\nduration, training time, and accuracy to prove that our method outperforms the\ntraditional methods.\n'

In [ ]:
recommendations(sentance,vectors)

  0%|          | 0/100000 [00:00<?, ?it/s]

NameError: ignored

The 'processed_data' used in the cell below was obtained after the kaggle notebook processing.


In [ ]:
processed_data = pd.read_csv('/content/drive/My Drive/processed_data.csv')
processed_data.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,title,abstract,year,authors,category,processed_text
0,704.0047,Intelligent location of simultaneously active ...,The intelligent acoustic emission locator is...,2009,T. Kosel and I. Grabec,cs.NE cs.AI,intelligent location simultaneously active aco...
1,704.005,Intelligent location of simultaneously active ...,Part I describes an intelligent acoustic emi...,2007,T. Kosel and I. Grabec,cs.NE cs.AI,intelligent location simultaneously active aco...
2,704.0304,The World as Evolving Information,This paper discusses the benefits of describ...,2013,Carlos Gershenson,cs.IT cs.AI math.IT q-bio.PE,world evolving information paper discusses ben...
3,704.0671,Learning from compressed observations,The problem of statistical learning is to co...,2016,Maxim Raginsky,cs.IT cs.LG math.IT,learning compressed observations problem stati...
4,704.0954,Sensor Networks with Random Links: Topology De...,"In a sensor network, in practice, the commun...",2009,Soummya Kar and Jose M. F. Moura,cs.IT cs.LG math.IT,sensor networks random links topology design d...


In [ ]:
processed_data['split_cat'] = [x.split() for x in processed_data['category']]

processed_data.head()

,id,title,abstract,year,authors,category,processed_text,split_cat
0,704.0047,Intelligent location of simultaneously active ...,The intelligent acoustic emission locator is...,2009,T. Kosel and I. Grabec,cs.NE cs.AI,intelligent location simultaneously active aco...,"[cs.NE, cs.AI]"
1,704.005,Intelligent location of simultaneously active ...,Part I describes an intelligent acoustic emi...,2007,T. Kosel and I. Grabec,cs.NE cs.AI,intelligent location simultaneously active aco...,"[cs.NE, cs.AI]"
2,704.0304,The World as Evolving Information,This paper discusses the benefits of describ...,2013,Carlos Gershenson,cs.IT cs.AI math.IT q-bio.PE,world evolving information paper discusses ben...,"[cs.IT, cs.AI, math.IT, q-bio.PE]"
3,704.0671,Learning from compressed observations,The problem of statistical learning is to co...,2016,Maxim Raginsky,cs.IT cs.LG math.IT,learning compressed observations problem stati...,"[cs.IT, cs.LG, math.IT]"
4,704.0954,Sensor Networks with Random Links: Topology De...,"In a sensor network, in practice, the commun...",2009,Soummya Kar and Jose M. F. Moura,cs.IT cs.LG math.IT,sensor networks random links topology design d...,"[cs.IT, cs.LG, math.IT]"


In [ ]:
processed_data.to_csv('processed_data2.csv',index=False)

In [ ]:
processed_data['category'].value_counts()

cs.LG      73630
cs.CV      64358
cs.AI      27233
stat.ML    17691
cs.RO      13803
cs.HC       7681
cs.NE       6031
Name: category, dtype: int64

Later in the project it was noticed that applying singular value decomposition on the complete data of 210k rows lead to the crashing of google colab instance due to insuffucient RAM. 

Therefore I had to sample 100k rows from the original data and work on them for the recommender.

In [ ]:
data = pd.read_csv("/content/drive/My Drive/final_data_100k.csv")
data.shape

(100000, 8)

In [ ]:
# data = data.sample(n=100000)

In [ ]:
import pickle
filename = 'tfidf_model'
outfile = open(filename,'wb')

In [ ]:
tfidf = TfidfVectorizer()        # Vectorizing the document using TF-IDF
vectors_tfidf = tfidf.fit_transform(data['processed_text'])
vectors_tfidf.shape
pickle.dump(tfidf, outfile)
outfile.close()

In [ ]:
pca = TruncatedSVD(n_components=2000)  # Applying SVD to find the top 2000 features in order to reduce dimensionality and 
vectors = pca.fit_transform(vectors_tfidf)  # reduce the time taken to calculate similarity.
print(vectors.shape)
np.save('vectors2',vectors)
filename2 = 'pcamodel'
outfile = open(filename2,'wb')
pickle.dump(pca,outfile)
outfile.close()

(100000, 2000)


In [ ]:
data.reset_index(inplace=True)

In [ ]:
data.to_csv("final_data_100k.csv",index=False)

As the google colab instance was crashing continuously. I had to save the models and outputs at every stage of the project.

In [4]:
vectors = np.load('/content/drive/My Drive/vectors.npy')

In [5]:
vectors.shape

(100000, 2000)

In [ ]:
data = pd.read_csv("/content/drive/My Drive/final_data_100k.csv")
data.head()

,index,id,title,abstract,year,authors,category,processed_text
0,173901,2107.08246,Woodscape Fisheye Semantic Segmentation for Au...,We present the WoodScape fisheye semantic se...,2021,"Saravanabalagi Ramachandran, Ganesh Sistu, Joh...",cs.CV,woodscape fisheye semantic segmentation autono...
1,63942,1809.04487,Discovering Topical Interactions in Text-based...,Social media conversations unfold based on c...,2018,"Srikanta Bedathur, Indrajit Bhattacharya, Jaye...",cs.LG,discovering topical interactions text based ca...
2,179,707.1193,Singular curves and cusp points in the joint s...,This paper investigates the singular curves ...,2007,"Mazen Zein (IRCCyN), Philippe Wenger (IRCCyN),...",cs.RO,singular curves cusp points joint space parall...
3,427,712.0189,Summarization and Classification of Non-Poisso...,Fitting models for non-Poisson point process...,2007,Jeffrey Picka and Mingxia Deng,stat.ML,summarization classification poisson point pro...
4,133972,2009.09169,BargainNet: Background-Guided Domain Translati...,Image composition is a fundamental operation...,2021,"Wenyan Cong, Li Niu, Jianfu Zhang, Jing Liang,...",cs.CV,bargainnet background guided domain translatio...


In [ ]:
data.shape

(100000, 8)

Using cosine similarity from 'numba' as it is several times faster than the sklearn implementation of cosine similarity.

In [6]:
from numba import jit

@jit(nopython=True)
def cosine_similarity_numba(u:np.ndarray, v:np.ndarray):
    assert(u.shape[0] == v.shape[0])
    uv = 0
    uu = 0
    vv = 0
    for i in range(u.shape[0]):
        uv += u[i]*v[i]
        uu += u[i]*u[i]
        vv += v[i]*v[i]
    cos_theta = 1
    if uu!=0 and vv!=0:
        cos_theta = uv/np.sqrt(uu*vv)
    return cos_theta

In [7]:
sims = pd.DataFrame(columns=list('abcdefghijklmnopqrst'))

The cell below was used for the finally computing the recommendations for the project. It iteratively computes similarity for a vector with all other vectors and saves the top 20 most similar at the corresponding index into another dataframe. The dataframe if downloaded into my system's memory after every 5000 iterations to avoid the loss of progress if the colab notebook disconnects or crashes.

The cell took a total of 14 hrs to run. It took 2 days for me to properly run this cell and download all the processed data without any issues as the colab notebook kept disconnecting.

In [8]:
# similarities = []
for item in tqdm(range(50001,100000)):
  cosine_sims = []
  for i in range(0,100000):
    cos = cosine_similarity_numba(vectors[item],vectors[i])
    cosine_sims.append(cos)
  sims.loc[len(sims)] = np.argsort(cosine_sims)[-22:-2]
  if (item%5000 == 0 or item == 99999) and item != 0:
    filename = 'top_recs3_'+str(item)+'.csv'
    sims.to_csv(filename,index=False)
    files.download(filename)
    print('Shape of ',filename,": ",sims.shape)

  0%|          | 0/49999 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Shape of  top_recs3_55000.csv :  (5000, 20)


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Shape of  top_recs3_95000.csv :  (45000, 20)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Shape of  top_recs3_99999.csv :  (49999, 20)


In [12]:
d1 = pd.read_csv('top_recs2_50000.csv')
d1.shape

(40000, 20)

In [13]:
d2 = pd.read_csv('top_recs3_99999.csv')
d2.shape

(49999, 20)

In [19]:
final_data = pd.concat([d0,d1,d2])
final_data.shape

(100000, 20)

In [22]:
final_data.reset_index(drop=True,inplace=True)
final_data.tail()

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t
99995,65902,27237,90946,38863,77160,39462,87396,78036,14765,25281,43185,67066,12989,64161,80929,26016,94034,97077,60469,96647
99996,67600,56383,67853,78873,13493,62343,11722,44790,98712,54390,39216,86626,65350,86443,90475,73094,17954,44694,53438,6040
99997,18719,16599,67714,90932,68924,23270,31844,94601,6822,30630,57624,30025,13250,91581,58122,83214,46160,92618,73925,9641
99998,98861,25188,90053,35128,1288,46070,37360,2147,84087,60266,57080,61341,72670,60167,88870,29093,64887,50444,13381,30742
99999,90602,49707,17058,16344,64607,19105,69252,7758,75266,69963,87070,20164,75193,45156,68638,81806,77637,41250,71937,97895


In [23]:
final_data.to_csv('final_recoms.csv',index=False)

In [24]:
final = pd.read_csv('final_recoms.csv')
final.head()

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t
0,84239,18062,71940,522,58187,19231,25975,33514,18267,23795,67384,15932,42819,87959,31634,48470,54490,67123,50283,76915
1,45784,31939,15113,74393,49569,64955,56262,48252,13196,9075,35925,98164,50280,93391,59935,49059,16166,42492,58783,35773
2,60380,52263,32145,184,71634,53394,1876,13427,44443,76284,92562,95303,52499,42098,35993,10496,76928,77138,63149,42391
3,63915,44786,82757,60271,60734,61705,9929,86120,35864,47912,44662,69829,60700,95647,58926,35150,52340,67012,26172,43247
4,34650,3738,72515,99572,73006,8369,70815,18214,25263,57975,3588,83675,80247,22888,34060,19066,36047,32163,19449,67638


In [25]:
data = pd.read_csv('/content/drive/MyDrive/final_data_100k.csv')
data.head()

,index,id,title,abstract,year,authors,category,processed_text
0,173901,2107.08246,Woodscape Fisheye Semantic Segmentation for Au...,We present the WoodScape fisheye semantic se...,2021,"Saravanabalagi Ramachandran, Ganesh Sistu, Joh...",cs.CV,woodscape fisheye semantic segmentation autono...
1,63942,1809.04487,Discovering Topical Interactions in Text-based...,Social media conversations unfold based on c...,2018,"Srikanta Bedathur, Indrajit Bhattacharya, Jaye...",cs.LG,discovering topical interactions text based ca...
2,179,707.1193,Singular curves and cusp points in the joint s...,This paper investigates the singular curves ...,2007,"Mazen Zein (IRCCyN), Philippe Wenger (IRCCyN),...",cs.RO,singular curves cusp points joint space parall...
3,427,712.0189,Summarization and Classification of Non-Poisso...,Fitting models for non-Poisson point process...,2007,Jeffrey Picka and Mingxia Deng,stat.ML,summarization classification poisson point pro...
4,133972,2009.09169,BargainNet: Background-Guided Domain Translati...,Image composition is a fundamental operation...,2021,"Wenyan Cong, Li Niu, Jianfu Zhang, Jing Liang,...",cs.CV,bargainnet background guided domain translatio...


In [26]:
final.head()

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t
0,84239,18062,71940,522,58187,19231,25975,33514,18267,23795,67384,15932,42819,87959,31634,48470,54490,67123,50283,76915
1,45784,31939,15113,74393,49569,64955,56262,48252,13196,9075,35925,98164,50280,93391,59935,49059,16166,42492,58783,35773
2,60380,52263,32145,184,71634,53394,1876,13427,44443,76284,92562,95303,52499,42098,35993,10496,76928,77138,63149,42391
3,63915,44786,82757,60271,60734,61705,9929,86120,35864,47912,44662,69829,60700,95647,58926,35150,52340,67012,26172,43247
4,34650,3738,72515,99572,73006,8369,70815,18214,25263,57975,3588,83675,80247,22888,34060,19066,36047,32163,19449,67638


In [40]:
data['title'][4]

'BargainNet: Background-Guided Domain Translation for Image Harmonization'

In [41]:
data['title'][34650]

'Full Reference Objective Quality Assessment for Reconstructed Background\n  Images'

In [42]:
result = pd.concat([data,final], axis=1)
result.head()

,index,id,title,abstract,year,authors,category,processed_text,a,b,...,k,l,m,n,o,p,q,r,s,t
0,173901,2107.08246,Woodscape Fisheye Semantic Segmentation for Au...,We present the WoodScape fisheye semantic se...,2021,"Saravanabalagi Ramachandran, Ganesh Sistu, Joh...",cs.CV,woodscape fisheye semantic segmentation autono...,84239,18062,...,67384,15932,42819,87959,31634,48470,54490,67123,50283,76915
1,63942,1809.04487,Discovering Topical Interactions in Text-based...,Social media conversations unfold based on c...,2018,"Srikanta Bedathur, Indrajit Bhattacharya, Jaye...",cs.LG,discovering topical interactions text based ca...,45784,31939,...,35925,98164,50280,93391,59935,49059,16166,42492,58783,35773
2,179,707.1193,Singular curves and cusp points in the joint s...,This paper investigates the singular curves ...,2007,"Mazen Zein (IRCCyN), Philippe Wenger (IRCCyN),...",cs.RO,singular curves cusp points joint space parall...,60380,52263,...,92562,95303,52499,42098,35993,10496,76928,77138,63149,42391
3,427,712.0189,Summarization and Classification of Non-Poisso...,Fitting models for non-Poisson point process...,2007,Jeffrey Picka and Mingxia Deng,stat.ML,summarization classification poisson point pro...,63915,44786,...,44662,69829,60700,95647,58926,35150,52340,67012,26172,43247
4,133972,2009.09169,BargainNet: Background-Guided Domain Translati...,Image composition is a fundamental operation...,2021,"Wenyan Cong, Li Niu, Jianfu Zhang, Jing Liang,...",cs.CV,bargainnet background guided domain translatio...,34650,3738,...,3588,83675,80247,22888,34060,19066,36047,32163,19449,67638


In [ ]:
result.to_csv('data_with_recoms.csv',index=False)

Finally all the partially downloaded recommendations were combined and downloaded for building the frontend using streamlit.